In [1]:
import numpy as np
import open3d as o3d
import correct_lidar

import glob

import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd
import datetime as dt

In [2]:
lidar_base_path = "/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/"

# Open an lvx file, convert to ply file (which cloud compare can use)

### For specific date

In [ ]:
specific_date_path = "/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/l1/2022-12-12/*.lvx"
files = glob.glob(specific_date_path)

In [ ]:
# get all files in the data/l2/snow_off directory
lidar = "l1"
for file in sorted(files)[-1:]:
    points = correct_lidar.load_first_file(lidar, file_search=file)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points.T)
    o3d.io.write_point_cloud(
        filename=file.replace(".lvx", ".ply"), 
        pointcloud=pcd
    )
ex_pc = file.replace(".lvx", ".ply")

In [ ]:
pcd = o3d.io.read_point_cloud(ex_pc)
o3d.visualization.draw_geometries([pcd])

## Open up scans from 1 lidar, taken at two different times

In [ ]:
date = "2022-12-12"
hour = "23"
minute = "55"

loaded = []
for l_n in [
    "l1", 
    # "l2", "l3", "l4", 
    # "l6"
]:
    new_points = correct_lidar.load_date(
            l_n, 
            date, 
            hour=hour, 
            minute=minute, 
            path=lidar_base_path, 
            load_all=True
        )
    new_points = new_points.T
    loaded.append(
        new_points   
    )
combined_points_bs = np.concatenate(loaded)

date = "2022-12-12"
hour = "00"
minute = "00"

loaded = []
for l_n in [
    "l1", 
    # "l2", "l3", "l4", 
    # "l6"
]:
    new_points = correct_lidar.load_date(
            l_n, 
            date, 
            hour=hour, 
            minute=minute, 
            path=lidar_base_path, 
            load_all=True
        )
    new_points = new_points.T
    loaded.append(
        new_points   
    )
combined_points_clean = np.concatenate(loaded)

In [ ]:
import pandas as pd
columns_convert = {0: 'x', 1: 'y', 2: 'z'}
df = pd.concat([
    pd.DataFrame(combined_points_clean).rename(columns=columns_convert).assign(type = 'clear event'),
    pd.DataFrame(combined_points_bs).rename(columns=columns_convert).assign(type = 'bs event')
])

In [ ]:
# Bin heights, using 10cm vertical spacing
df['z_bin'] = pd.cut(
    df['z'], 
    np.linspace(-30, 1, 31*10+1)
)
df['z_bin'] = df['z_bin'].apply(lambda x: (x.left + x.right)/2)
df_bin_count = pd.DataFrame(df.groupby(['z_bin', 'type']).size()).rename(columns={0:'count'}).reset_index()

# Calculate the difference in bin counts between scans during BS and clear weather
df_binned_subtracted = df_bin_count.pivot(index='z_bin', columns='type', values='count')
df_binned_subtracted['absolute'] = df_binned_subtracted['bs event'] - df_binned_subtracted['clear event']
df_binned_subtracted = df_binned_subtracted.reset_index()
df_binned_subtracted['z_bin'] = df_binned_subtracted['z_bin'].astype('float')

In [ ]:
alt.Chart(df_bin_count.query("count > 0")).mark_line(point=True).encode(
    alt.X("count:Q").sort('-y').scale(type='log').title("number of points, 10cm vertical bins"),
    alt.Y("z_bin:Q").title("height (m)"),
    alt.Color("type:N")
).configure_legend(
    orient='bottom-right', fillColor='white'
).properties(width=250, height = 250).display(renderer='svg')

In [ ]:
alt.Chart(
    df_binned_subtracted.query("absolute > 0")).mark_line(point=True).encode(
    alt.X("absolute:Q").sort('-y').scale(type='log'),
    alt.Y("z_bin:Q"),
).properties(width=250, height = 250).display(renderer='svg')

In [ ]:
alt.Chart(
    df_binned_subtracted.query("absolute > 0").query("z_bin > -9").query("absolute > 10")).mark_line(point=True).encode(
    alt.X("absolute:Q").sort('-y').scale(type='log'),
    alt.Y("z_bin:Q"),
).properties(width=250, height = 250).display(renderer='svg')

## Open up scans from all lidars, at 2 different times

In [ ]:
date = "2022-12-12"
hour = "23"
minute = "55"

loaded = []
for l_n in [
    "l1", "l2", "l3", "l6", # "l4",
]:
    new_points = correct_lidar.load_date(
            l_n, 
            date, 
            hour=hour, 
            minute=minute, 
            path=lidar_base_path, 
            load_all=True
        )
    new_points = new_points.T
    loaded.append(
        new_points   
    )
combined_points_bs = np.concatenate(loaded)

date = "2022-12-12"
hour = "00"
minute = "00"

loaded = []
for l_n in [
    "l1", "l2", "l3", "l6", # "l4",
]:
    new_points = correct_lidar.load_date(
            l_n, 
            date, 
            hour=hour, 
            minute=minute, 
            path=lidar_base_path, 
            load_all=True
        )
    new_points = new_points.T
    loaded.append(
        new_points   
    )
combined_points_clean = np.concatenate(loaded)

#### Visualize the point clouds using open 3d

In [ ]:
# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
pcd_bs = o3d.geometry.PointCloud()
pcd_bs.points = o3d.utility.Vector3dVector(combined_points_bs)

pcd_clean = o3d.geometry.PointCloud()
pcd_clean.points = o3d.utility.Vector3dVector(combined_points_clean)

In [ ]:
o3d.visualization.draw_geometries([pcd_bs])

In [ ]:
o3d.visualization.draw_geometries([pcd_clean])

#### Visualize with vertical binning

In [ ]:
columns_convert = {0: 'x', 1: 'y', 2: 'z'}
df = pd.concat([
    pd.DataFrame(combined_points_clean).rename(columns=columns_convert).assign(type = 'clear event'),
    pd.DataFrame(combined_points_bs).rename(columns=columns_convert).assign(type = 'bs event')
])

In [ ]:
# Bin heights, using 10cm vertical spacing
df['z_bin'] = pd.cut(
    df['z'], 
    np.linspace(-30, 1, 31*10+1)
)
df['z_bin'] = df['z_bin'].apply(lambda x: (x.left + x.right)/2)
df_bin_count = pd.DataFrame(df.groupby(['z_bin', 'type']).size()).rename(columns={0:'count'}).reset_index()

# Calculate the difference in bin counts between scans during BS and clear weather
df_binned_subtracted = df_bin_count.pivot(index='z_bin', columns='type', values='count')
df_binned_subtracted['absolute'] = df_binned_subtracted['bs event'] - df_binned_subtracted['clear event']
df_binned_subtracted = df_binned_subtracted.reset_index()
df_binned_subtracted['z_bin'] = df_binned_subtracted['z_bin'].astype('float')

In [ ]:
alt.Chart(df_bin_count.query("count > 0")).mark_line(point=True).encode(
    alt.X("count:Q").sort('-y').scale(type='log').title("number of points, 10cm vertical bins"),
    alt.Y("z_bin:Q").title("height (m)"),
    alt.Color("type:N")
).configure_legend(
    orient='bottom-right', fillColor='white'
).properties(width=250, height = 250).display(renderer='svg')

In [ ]:
alt.Chart(
    df_binned_subtracted.query("absolute > 0")).mark_line(point=True).encode(
    alt.X("absolute:Q").sort('-y').scale(type='log'),
    alt.Y("z_bin:Q"),
).properties(width=250, height = 250).display(renderer='svg')

In [ ]:
alt.Chart(
    df_binned_subtracted.query("absolute > 0").query("z_bin > -9").query("absolute > 10")).mark_line(point=True).encode(
    alt.X("absolute:Q").sort('-y').scale(type='log'),
    alt.Y("z_bin:Q"),
).properties(width=250, height = 250).display(renderer='svg')

## Open up scans from all lidars, for a time series of times, saving only vertically averaged data along the way

In [182]:
columns_convert = {0: 'x', 1: 'y', 2: 'z'}

df_list = []

# iterate over timestamps from a single day
for hour in range(0,24):
    print(f"{hour} of 24 hours")
    for minute in np.linspace(0,55, 12):
        print(f"{int(minute)} of 55 minutes")

        all_paths_this_timestamp = glob.glob(
            f"/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/*/2022-12-12/2022-12-12_{hour:02}-{int(minute):02}-*.lvx"
        )
        # REMOVE SCANS FROM L4
        all_paths_this_timestamp = [file for file in all_paths_this_timestamp if '/l4/' not in file]
        assert len(all_paths_this_timestamp) == 4
        # Load points from all scans during this timestamp
        loaded_points = []
        for path in all_paths_this_timestamp:
            lidar_str = path.split('lidar_raw/')[1].split('/')[0]
            points = correct_lidar.load_file(path, lidar_str)
            points = points.T
            loaded_points.append(
                points   
            )
        combined_points = np.concatenate(loaded_points)

        # Put points into a dataframe, assign time columns
        df = pd.DataFrame(combined_points).rename(columns=columns_convert)
        # Separate into 10cm vertical bins, count points
        df['z_bin'] = pd.cut(
            df['z'], 
            np.linspace(-30, 1, 31*10+1)
        )
        df['z_bin'] = df['z_bin'].apply(lambda x: (x.left + x.right)/2)
        df_bin_count = pd.DataFrame(df.groupby('z_bin').size()).rename(columns={0:'count'}).reset_index()
        this_datetime = dt.datetime(
            2022, 
            12, 
            12, 
            hour, 
            int(minute)
        )
        df_bin_count = df_bin_count.assign(time = this_datetime)
        df_list.append(df_bin_count)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_33173/3014154163.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_bin_count = pd.DataFrame(df.groupby('z_bin').size()).rename(columns={0:'count'}).reset_index()


55 of 55 minutes


/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_33173/3014154163.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_bin_count = pd.DataFrame(df.groupby('z_bin').size()).rename(columns={0:'count'}).reset_index()


2 of 24 hours
0 of 55 minutes


/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_33173/3014154163.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_bin_count = pd.DataFrame(df.groupby('z_bin').size()).rename(columns={0:'count'}).reset_index()


...

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_33173/3014154163.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_bin_count = pd.DataFrame(df.groupby('z_bin').size()).rename(columns={0:'count'}).reset_index()


50 of 55 minutes


/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_33173/3014154163.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_bin_count = pd.DataFrame(df.groupby('z_bin').size()).rename(columns={0:'count'}).reset_index()


55 of 55 minutes


/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_33173/3014154163.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_bin_count = pd.DataFrame(df.groupby('z_bin').size()).rename(columns={0:'count'}).reset_index()


In [183]:
df = pd.concat(df_list)

In [188]:
df.to_parquet("all_day_binned_point_counts.parquet")

In [163]:
df = pd.read_parquet("all_day_binned_point_counts.parquet")

In [164]:
from sublimpy import utils

In [165]:
df = utils.modify_df_timezone(df, 'UTC', 'US/Mountain')

In [166]:
df['minutes'] = df['time'].dt.minute
df['hours'] = df['time'].dt.hour

In [167]:
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [168]:
df = df[df.time.dt.day == 12]

In [170]:
alt.Chart(
    df.query("z_bin > -10")
).mark_line().encode(
    alt.X("count:Q").scale(type='symlog').sort('-y').axis(values=[0, 1,100,10000,10000]),
    alt.Y("z_bin:Q"),
    alt.Color("minutes:O"),
    alt.Facet("hours:O", columns=9)
).properties(width = 100, height = 100)

alt.Chart(...)

#### bring in the sos tidy data

In [142]:
# Open SOS Measurement Dataset
################################################
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df = pd.read_parquet(f'../paper1/tidy_df_{start_date}_{end_date}_noplanar_fit.parquet')

# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')
tidy_df = tidy_df[
    (tidy_df.time >= df.time.min())
    &
    (tidy_df.time <= df.time.max())
]

In [143]:
alt.Chart(
    tidy_df[tidy_df.variable.isin(['SF_avg_1m_ue', 'SF_avg_2m_ue'])]
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q"),    
    alt.Color("height:N")
)

alt.Chart(...)

#### Calculate the profiles used to remove physical obstructions

In [144]:
profile_df = df.query("z_bin > -8")

In [145]:
obstruction_normalize = profile_df[profile_df['time'] < "2022-12-12 05:00:00"]
obstruction_normalize = obstruction_normalize.groupby('z_bin')[['count']].median()
obstruction_normalize

,count
z_bin,
-7.95,1363.5
-7.85,998.5
-7.75,966.5
-7.65,994.0
-7.55,975.0
...,...
0.55,0.0
0.65,0.0
0.75,1.0


#### Remove them

In [146]:
physical_obstructions_removed = profile_df.groupby(["time", "hours", "minutes"]).apply(lambda df: df.set_index('z_bin') - obstruction_normalize).drop(columns=['time', 'hours', 'minutes']).reset_index()
physical_obstructions_removed

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_35842/3592351243.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  physical_obstructions_removed = profile_df.groupby(["time", "hours", "minutes"]).apply(lambda df: df.set_index('z_bin') - obstruction_normalize).drop(columns=['time', 'hours', 'minutes']).reset_index()


,time,hours,minutes,z_bin,count
0,2022-12-12 00:00:00,0,0,-7.95,25.5
1,2022-12-12 00:00:00,0,0,-7.85,62.5
2,2022-12-12 00:00:00,0,0,-7.75,17.5
3,2022-12-12 00:00:00,0,0,-7.65,-89.0
4,2022-12-12 00:00:00,0,0,-7.55,-42.0
...,...,...,...,...,...
18355,2022-12-12 16:55:00,16,55,0.55,0.0
18356,2022-12-12 16:55:00,16,55,0.65,1.0
18357,2022-12-12 16:55:00,16,55,0.75,-1.0
18358,2022-12-12 16:55:00,16,55,0.85,-1.0


In [147]:
alt.Chart(
    physical_obstructions_removed
).mark_line().encode(
    alt.X("count:Q").scale(type='linear').sort('-y'),
    alt.Y("z_bin:Q"),
    alt.Color("minutes:O"),
    alt.Facet("hours:O", columns=6)
).properties(width = 100, height = 100)

alt.Chart(...)

#### Vertically integrate point concentrations

In [148]:
physical_obstructions_removed_integrated = physical_obstructions_removed.groupby('time')[['count']].sum().reset_index()

In [161]:
(alt.Chart(
    physical_obstructions_removed_integrated
).mark_line().encode(
    alt.X("time:T").axis(title=None, labels=False),
    alt.Y("count:Q").title(["Vertically-integrated", "point count"])
).properties(height = 100) &\
alt.Chart(
    tidy_df[tidy_df.variable.isin(['SF_avg_1m_ue', 'SF_avg_2m_ue'])]
).mark_line().encode(
    alt.X("time:T").axis(title=None, labels=False),
    alt.Y("value:Q").title(["FlowCapt blowing", "snow flux (g/m^2/s)"]),
    alt.Color("height:N")
).properties(height = 100) &\
alt.Chart(
    tidy_df[tidy_df.variable.isin(['ldiag_10m_ue', 'ldiag_10m_uw', 'ldiag_10m_c', 'ldiag_10m_c'])]
).mark_line().encode(
    alt.X("time:T").axis(title=None, labels=False),
    alt.Y("value:Q").title(["Sonic anemometer", "quality flag"]),
    alt.Color("tower:N")
).properties(height = 100)).resolve_scale(color='independent')  &\
alt.Chart(
    tidy_df[tidy_df.variable.isin(['w_h2o__10m_ue', 'w_h2o__10m_uw', 'w_h2o__10m_c', 'w_h2o__10m_c'])]
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Water vapor flux (g/m^2/s)"),    
    alt.Color("tower:N")
).properties(height = 100).resolve_scale(x = 'shared', color='independent')

alt.VConcatChart(...)